[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neuromatch/climate-course-content/blob/main/tutorials/W1D5_ClimateModeling/W1D5_Tutorial7.ipynb) &nbsp; <a href="https://kaggle.com/kernels/welcome?src=https://raw.githubusercontent.com/neuromatch/climate-course-content/main/tutorials/W1D5_ClimateModeling/W1D5_Tutorial7.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open in Kaggle"/></a>

# Bonus Tutorial 10: Surface energy balance in an ESM


**Week 1, Day 5, Climate Modeling**

**Content creators:** Brodie Pearson, Jenna Pearson, Julius Busecke, Tom Nicholas, and Brian E. J. Rose

**Content reviewers:** Yunlong Xu, Will Gregory, Peter Ohue, Derick Temfack, Zahra Khodakaramimaghsoud, Peizhen Yang, Younkap Nina Duplex, Ohad Zivan, Chi Zhang

**Content editors:** Abigail Bodner, Ohad Zivan, Chi Zhang

**Production editors:** Wesley Banfield, Jenna Pearson, Chi Zhang, Ohad Zivan

**Our 2024 Sponsors:** NASA TOPS, Google DeepMind, CMIP, NFDI4Earth



# Tutorial Objectives

By the end of this tutorial students will be able to:
* Load atmospheric data from the Community Earth System Model (CESM), which was used in the most recent [Coupled Model Intercomparison Project (CMIP6)](https://wcrp-cmip.org/cmip-phase-6-cmip6/)
* Analyze the zonal-mean surface energy budget of a realistic climate model (i.e., the budget at each latitude)
* Link variations in different surface heat flux to physical differences in the air-surface conditions across regions.


# Setup

In [ ]:
# installations ( uncomment and run this cell ONLY when using google colab or kaggle )

# note the conda install takes quite a while, but conda is REQUIRED to properly download the
# dependencies (that are not just python packages)

# !pip install condacolab &> /dev/null
# import condacolab
# condacolab.install()

# crucial to install all packages in one line, otherwise code will fail.
# !mamba install xarray-datatree intake-esm gcsfs xmip aiohttp cartopy nc-time-axis cf_xarray xarrayutils &> /dev/null

In [ ]:
# imports

# google colab users, if you get an error please run this cell again

import intake
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

from xmip.preprocessing import combined_preprocessing
from xarrayutils.plotting import shaded_line_plot
from xmip.utils import google_cmip_col

from datatree import DataTree
from xmip.postprocessing import _parse_metric

import cartopy.crs as ccrs

In [ ]:
# @title Figure Settings
import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle")

In [ ]:
# @title Video 1: Introduction to Earth System Models

from ipywidgets import widgets
from IPython.display import YouTubeVideo
from IPython.display import IFrame
from IPython.display import display


class PlayVideo(IFrame):
  def __init__(self, id, source, page=1, width=400, height=300, **kwargs):
    self.id = id
    if source == 'Bilibili':
      src = f'https://player.bilibili.com/player.html?bvid={id}&page={page}'
    elif source == 'Osf':
      src = f'https://mfr.ca-1.osf.io/render?url=https://osf.io/download/{id}/?direct%26mode=render'
    super(PlayVideo, self).__init__(src, width, height, **kwargs)


def display_videos(video_ids, W=400, H=300, fs=1):
  tab_contents = []
  for i, video_id in enumerate(video_ids):
    out = widgets.Output()
    with out:
      if video_ids[i][0] == 'Youtube':
        video = YouTubeVideo(id=video_ids[i][1], width=W,
                             height=H, fs=fs, rel=0)
        print(f'Video available at https://youtube.com/watch?v={video.id}')
      else:
        video = PlayVideo(id=video_ids[i][1], source=video_ids[i][0], width=W,
                          height=H, fs=fs, autoplay=False)
        if video_ids[i][0] == 'Bilibili':
          print(f'Video available at https://www.bilibili.com/video/{video.id}')
        elif video_ids[i][0] == 'Osf':
          print(f'Video available at https://osf.io/{video.id}')
      display(video)
    tab_contents.append(out)
  return tab_contents


video_ids = [('Youtube', 'e3O9DmhE46Y'), ('Bilibili', 'BV1qh4y1G7EC')]
tab_contents = display_videos(video_ids, W=730, H=410)
tabs = widgets.Tab()
tabs.children = tab_contents
for i in range(len(tab_contents)):
  tabs.set_title(i, video_ids[i][0])
display(tabs)

In [ ]:
# @markdown
from ipywidgets import widgets
from IPython.display import IFrame

link_id = "dn8hs"

download_link = f"https://osf.io/download/{link_id}/"
render_link = f"https://mfr.ca-1.osf.io/render?url=https://osf.io/{link_id}/?direct%26mode=render%26action=download%26mode=render"
# @markdown
out = widgets.Output()
with out:
    print(f"If you want to download the slides: {download_link}")
    display(IFrame(src=f"{render_link}", width=730, height=410))
display(out)

# Section 1: Atmospheric data from the Community Earth System Model (CESM)

In this tutorial, we will repeat the CMIP6 data access from the previous tutorial, but this time we will look at data from the _atmospheric_ component of the Community Earth System Model](https://www.cesm.ucar.edu/models/cesm2) (**CESM2**)

We will be analyzing output from a **historical simulation** of CESM2, which covers the period 1850 to 2015 using the historicallly observed forcing of the climate system.

## Section 1.1: Finding & Opening CMIP6 Data with Xarray

As a reminder, to access the data we will first load an [intake-esm](https://intake-esm.readthedocs.io/en/stable/) catalog object based on a CSV file that is maintained by the pangeo community. 

Then, we will search the catalog for our *facets* of interest. Here we will be using `table_id="Amon"` for monthly atmospheric model output. We will use a list of different `variable_id` values corresponding to physical components of the energy budget at Earth's surface.

In [ ]:
col = intake.open_esm_datastore(
    "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
)  # open an intake catalog containing the Pangeo CMIP cloud data


# from the full `col` object, create a subset using facet search
cat = col.search(
    source_id="CESM2",
    variable_id=[
        "hfls",
        "hfss",
        "rlds",
        "rlus",
        "rsds",
        "rsus",
        "tas",
    ],
    member_id="r1i1p1f1",
    table_id="Amon",
    grid_label="gn",
    experiment_id=["historical", "hist-nat"],
    require_all_on=[
        "source_id"
    ],  # make sure that we only get models which have all of the above experiments
)

# convert the sub-catalog into a datatree object, by opening each dataset into an Xarray.Dataset (without loading the data)
kwargs = dict(
    preprocess=combined_preprocessing,  # apply xMIP fixes to each dataset
    xarray_open_kwargs=dict(
        use_cftime=True
    ),  # ensure all datasets use the same time index
    storage_options={
        "token": "anon"
    },  # anonymous/public authentication to google cloud storage
)

cat.esmcat.aggregation_control.groupby_attrs = ["source_id", "experiment_id"]
dt = cat.to_datatree(**kwargs)

## Section 1.2: Checking the CMIP6 Datasets

As in the previous tutorial, we now have a "[datatree](https://xarray-datatree.readthedocs.io/en/latest/quick-overview.html)" containing the data we searched for:

In [ ]:
dt

Now, let's pull out relevant CESM2 datasets from the datatree; the *historical* simulation (human & natural forcing) and the *hist-nat* simulation (natural forcing only).

In [ ]:
# the historical including anthropogenic forcing
hist_dt = dt["CESM2"]["historical"]
hist_dt

In [ ]:
# the historical run without anthropogenic forcing
hist_nat = dt["CESM2"]["hist-nat"]
hist_nat

# Section 2:  Surface Energy Balance


Up until this point we have been analyzing budgets at the top of the atmosphere. Now we will move to Earth's surface, where we will run into both familiar and unfamiliar terminology. 

The first two terms we will analyze define the radiative component of the surface energy budget: shortwave and longwave radiation. For each of these terms, there is an upwelling and downwelling component. This is because, for example, some of the downwelling shortwave radiation is reflected back upwards due to the surface albedo. Similarly, some upwelling longwave radiation from Earth is absorbed by the atmosphere and re-emitted back towards the surface. The net radiative flux is given as 

\begin{align}
R_{net} = [R_{\uparrow S} - R_{\downarrow S}] + [R_{\uparrow L} - R_{\downarrow L}]
\end{align}

where the arrows indicate up(down)welling and the $S$ and $L$ are shortwave and longwave. Note that in keeping with the rest of the variables we will look at, the net radiation is defined so that a positive value refers to an upward flux of energy from the ocean or land to the atmosphere. That is, positive indicates heat transport upwards and away from the surface.

We will also be looking at latent and sensible heat. **[Sensible heat](https://glossary.ametsoc.org/wiki/Sensible_heat)** is the heat transferred due to a temperature difference between touching materials, for example between the air and the land or ocean surface. In this case, if the surface air is warmer than the land/ocean then heat is transferred from the air to the land/ocean (which is a downward or negative sensible heat flux), with an opposite-direction transfer if the air was colder than the land/ocean.

**[Latent heat](https://glossary.ametsoc.org/wiki/Latent_heat)** is due to evaporation and condensation of water, as these phase changes absorb and release heat respectively. Here 'latent' means hidden, in that the energy is stored in molecules and there is no temperature change sensed through a thermometer even though a heat transfer takes place.

While these are not the only terms that comprise the surface energy budget (i.e. geothermal heating, latent heat of fusion for melting ice or snow, biological processes) these are typically the dominant terms that set the global patterns.

Let's take a look at the `long_name` of some variables we just downloaded.


In [ ]:
(
    hist_dt.hfls.long_name,
    hist_dt.hfss.long_name,
    hist_dt.rlds.long_name,
    hist_dt.rlus.long_name,
    hist_dt.rsds.long_name,
    hist_dt.rsus.long_name,
)

In [ ]:
# predefine heat fluxes for each experiement, take annual means

# model output variables (.squeeze() removes singleton dimensions)
hist_am_latent_heat_flux = hist_dt.ds.hfls.mean(dim="time").squeeze()
hist_am_sensible_heat_flux = hist_dt.ds.hfss.mean(dim="time").squeeze()
hist_am_downwelling_longwave_flux = hist_dt.ds.rlds.mean(dim="time").squeeze()
hist_am_upwelling_longwave_flux = hist_dt.ds.rlus.mean(dim="time").squeeze()
hist_am_downwelling_shortwave_flux = hist_dt.ds.rsds.mean(dim="time").squeeze()
hist_am_upwelling_shortwave_flux = hist_dt.ds.rsus.mean(dim="time").squeeze()

# derived variables
hist_am_net_shortwave_heat_flux = (
    hist_am_upwelling_shortwave_flux - hist_am_downwelling_shortwave_flux
)
hist_am_net_longwave_heat_flux = (
    hist_am_upwelling_longwave_flux - hist_am_downwelling_longwave_flux
)
hist_am_net_heat_flux = (
    hist_am_net_shortwave_heat_flux
    + hist_am_net_longwave_heat_flux
    + hist_am_latent_heat_flux
    + hist_am_sensible_heat_flux
)

Now we will plot the annual mean over the historical time period 1850-2015.

In [ ]:
%matplotlib inline

fig, ([ax_latent, ax_shortwave], [ax_sensible, ax_longwave]) = plt.subplots(
    ncols=2, nrows=2, figsize=[12, 6], subplot_kw={"projection": ccrs.Robinson()}
)

# plot surface latent heat flux the first month of the historical period
hist_am_latent_heat_flux.plot(
    ax=ax_latent,
    x="lon",
    y="lat",
    transform=ccrs.PlateCarree(),
    vmin=-300,
    vmax=300,
    cmap="coolwarm",
    robust=True,
    cbar_kwargs={"label": "$W/m^2$"},
)
ax_latent.coastlines()
ax_latent.set_title("Latent Heat Flux")

# Repeat for sensible heat flux
hist_am_sensible_heat_flux.plot(
    ax=ax_sensible,
    x="lon",
    y="lat",
    transform=ccrs.PlateCarree(),
    vmin=-150,
    vmax=150,
    cmap="coolwarm",
    robust=True,
    cbar_kwargs={"label": "$W/m^2$"},
)
ax_sensible.coastlines()
ax_sensible.set_title("Sensible Heat Flux")

# Repeat for net shortwave radiative heat flux
hist_am_net_shortwave_heat_flux.plot(
    ax=ax_shortwave,
    x="lon",
    y="lat",
    transform=ccrs.PlateCarree(),
    vmin=-300,
    vmax=300,
    cmap="coolwarm",
    robust=True,
    cbar_kwargs={"label": "$W/m^2$"},
)
ax_shortwave.coastlines()
ax_shortwave.set_title("Net Upward Shortwave Flux")

# Repeat for net longwave radiative heat flux
hist_am_net_longwave_heat_flux.plot(
    ax=ax_longwave,
    x="lon",
    y="lat",
    transform=ccrs.PlateCarree(),
    vmin=-150,
    vmax=150,
    cmap="coolwarm",
    robust=True,
    cbar_kwargs={"label": "$W/m^2$"},
)
ax_longwave.coastlines()
ax_longwave.set_title("Net Upward Longwave Flux");

## Questions 2: Climate Connection

1. What do you think causes the large spatial variations of the sensible heat flux between strong positive, weak, and strong negative fluxes? Consider different surface types in your answer.

In [ ]:
# to_remove explanation

"""
1. Sensible heat fluxes depend on temperature differences between the Earth's surface and the surface air. The differences in the sensible heat flux seen here are largely caused by the different properties of the three main surface types: land, ocean, and ice. Land typically warms fast when exposed to insolation, the ocean has a much higher heat capacity so does not warm as much as land in response to insolation, and ice typically remains near zero degrees Celsius or colder. This means that land is often warmer than the surface air (strong positive sensible heat flux), the ocean and surface air often have a similar temperature (weak flux), and ice is often much cooler than surface air (strong negative flux)
""";

# Section 3: Surface Energy Budget by Latitude

We can also calculate a *zonal average* which allows us to compare the contributions of each of these fluxes to the net heat flux by latitude (similar to the plot in the last lecture that considered the RCE model prediction as a function of latitude).

To calculate a spatial average of a gridded data set, we often have to weight the data based on the size of the area it is describing. Fortunately, CESM data is on a regular latitude-longitude grid, which means that grid cells at a specific latitude have the same area as all the other grid cells at that latitude. This makes a zonal average easy, because at each latitude we can simply calculate the mean of all data at that latitude. 

*Note: Our averaging would have required area-weighting if we were calculating a global mean (as you did in previous Tutorials) or if you had irregularly gridded data (which we will encounter on W2D1)!*

In [ ]:
# find the zonal means (.squeeze() removes singleton dimensions)
hist_azm_latent_heat_flux = hist_am_latent_heat_flux.mean(dim="x").squeeze()
hist_azm_sensible_heat_flux = hist_am_sensible_heat_flux.mean(dim="x").squeeze()
hist_azm_net_shortwave_flux = hist_am_net_shortwave_heat_flux.mean(dim="x").squeeze()
hist_azm_net_longwave_flux = hist_am_net_longwave_heat_flux.mean(dim="x").squeeze()
hist_azm_net_heat_flux = hist_am_net_heat_flux.mean(dim="x").squeeze()

In [ ]:
lat = hist_am_latent_heat_flux.lat[0, :]

fig, ax = plt.subplots(figsize=(8, 5))

ax.plot(lat, hist_azm_latent_heat_flux, label="Latent Heat")
ax.plot(lat, hist_azm_sensible_heat_flux, label="Sensible Heat")
ax.plot(lat, hist_azm_net_shortwave_flux, label="Shortwave")
ax.plot(lat, hist_azm_net_longwave_flux, label="Longwave")
ax.plot(lat, hist_azm_net_heat_flux, lw=3, color="k", label="Net")
ax.plot(lat, 0 * lat, color="black")

ax.set_title("Components of Annual Surface Energy Budget (+ up)")
ax.legend()

## Questions 3: Climate Connection

1.  Describe the dominant balance in the tropics (30ºS to 30ºN).
2.  What about for the polar regions (above 60ºN and below 60ºS).
3.  What do you think causes the dip in latent heat near the equator?
4.  Is there symmetry in the longwave radiation in the high southern and northern latitudes? What about for sensible heat?

In [ ]:
# to_remove explanation

"""
1. The tropics are warmed by shortwave radiation, which is balanced by cooling from latent heat (evaporation), longwave radiation (because the surface is warm) and a smaller sensible heat flux. There is a net negative imbalance, so other processes (e.g., lateral movement of heat) must transport surface heat away from the tropics.
2. In polar regions the shortwave incoming radiation and longwave outgoing radiation are the dominant balance in the Antarctic, while the Arctic also has a significant downward flux sensible heat (the atmosphere is typically warmer than the ice/ocean/land).
3. The intertropical convergence zone. This is region of significant cloud cover and precipitation created by surface air from both hemispheres converging near the Equator. This is also part of the reason the other fluxes also show dips near the Equator.
4. Longwave radiation is relatively symmetric. Some differences may be cause by hemispheric differences in relative land/ocean cover at each latitude. Sensible heat is quite symmetric at most latitudes (small changes due to surface type similar to the longwave radiation), but there is a significant negative sensible heat flux in the Arctic due to the combination of cold ice/ocean near the North pole, and relatively warm air (versus the Antarctic continent where air is consistently very cold)
""";

# Bonus Question: Climate Connection

5. Hypothetically, imagine this plot showed that the net heat flux was:
  - Negative 100 W m$^{-2}$ between 45ºN and 45ºS (i.e., 90 degrees of latitude centered on the Equator) and,
  - Positive 100 W m$^{-2}$ from 45ºN to 90ºN and from 45ºS to 90ºS.

  Would you expect Earth to warm, cool, or remain the same temperature? Why? 
  

In [ ]:
# to_remove explanation

"""
5. Even though both latitude bands span 90 degrees, the poles of the Earth (highest 45 degrees of each hemisphere) cover a much smaller area than the Equatorial regions (lower 45 degrees of each hemisphere). Since the energy being entering or leaving the Earth system is the flux multiplied by the area, and the lower latitude region (negative/downward heat flux) covers a much larger area than the high latitude regions, the planet would be expected to warm in this situation.
""";

# Summary
In this bonus tutorial, you analyzed atmospheric data from a CESM simulation, focusing on how shortwave/longwave radiation, sensible and latent heat, contribute to the surface energy budget. This tutorial also explored the *zonal-mean* surface energy budget, elucidating how different surface heat flux components vary with latitude due to physical differences in air-surface conditions across regions.

# Resources

This tutorial uses data from the simulations conducted as part of the [CMIP6](https://wcrp-cmip.org/) multi-model ensemble. 

For examples on how to access and analyze data, please visit the [Pangeo Cloud CMIP6 Gallery](https://gallery.pangeo.io/repos/pangeo-gallery/cmip6/index.html) 

For more information on what CMIP is and how to access the data, please see this [page](https://github.com/ClimateMatchAcademy/course-content/blob/main/tutorials/CMIP/CMIP_resource_bank.md).